# MCL732 - ASSIGNMENT 1 (PM 2.5)

Creating :


*   weighting matrix (W)
*   Source Matrix (A)
*   Specie Contribution Matrix (C)





In [19]:
import pandas as pd
import numpy as np

def create_weighting_matrix(excel_path):
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Extract the standard deviation values from the 'SD' column
    standard_deviations = df['SD'].values

    # Create the diagonal matrix with entries as 1/(standard deviation)^2
    weighting_matrix = np.diag(1 / standard_deviations**2)

    return weighting_matrix

def create_source_matrix(excel_path):
    # Read the Excel file
    df = pd.read_excel(excel_path, index_col=0)  # Assuming species names are in the first column

    # Extract numerical values associated with different sources
    source_matrix = df.values

    return source_matrix

def create_vector_c(excel_path):
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Extract values from the second column
    vector_c = df.iloc[0:, 1].values

    # Convert the vector to a column vector
    vector_c = vector_c.reshape(-1, 1)

    return vector_c

def cmb_solution(A, W, C):
    # Calculate the solution using Equation 26.16
    ATWA_inv = np.linalg.inv(A.T @ W @ A)
    s = ATWA_inv @ A.T @ W @ C

    return s

# Excel paths to access data
excel_path_W = 'SD 2.5.xlsx' #this matrix contains standard deviations
excel_path_A = 'Matrix A.xlsx' #this contians the aij matrix
excel_path_C = 'C 2.5.xlsx' #this contians the total contribution of each specie
eq_26_17_excel_path = 'eq 26.17.xlsx' #this contains sigma_aij for V matrix
# Create the weighting matrix
W = create_weighting_matrix(excel_path_W)

# Create the source matrix
A = create_source_matrix(excel_path_A)

# Create the vector C
C = create_vector_c(excel_path_C)

# Apply the CMB solution
source_contributions = cmb_solution(A, W, C)

# Print the resulting matrices and vectors
print("Weighting Matrix (W):")
print(W)

print("\nSource Matrix (A):")
print(A)

print("\nVector C:")
print(C)

print("\nSource Contributions %:")
print(source_contributions * 100)
#####################################################################################################################################

Weighting Matrix (W):
[[7.65433630e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.73921028e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.60846809e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+

 Finding Different Sources for which data is available

In [20]:
#####################################################################################################################################
import pandas as pd

def extract_source_names(excel_path):
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Extract source names from the first row
    source_names = df.columns[1:].tolist()

    return source_names

excel_path = 'Matrix A.xlsx'

# Extract source names
source_names = extract_source_names(excel_path)

# Print the extracted source names
print("Source Names:")
print(source_names)
#####################################################################################################################################

Source Names:
['Paved Road Dust', 'Vegetative Burning', 'Primary Crude Oil', 'Motor Vehicles', 'Limestone', 'MARINE', '(NH4)2SO4', 'NH4NO3', 'Secondary OC', 'NaNO3']


# Calculationg Source Profiles for Matrix W

In [21]:
source = extract_source_names('Matrix A.xlsx')
for i in range(len(source)):
  print(source[i],"-",source_contributions[i][0] * 100)

Paved Road Dust - 1.2971225183136013
Vegetative Burning - 7.001233634333547
Primary Crude Oil - 0.33061145846099277
Motor Vehicles - 8.622700863272355
Limestone - 0.0832866286534129
MARINE - 0.07128655971071106
(NH4)2SO4 - 3.1612843372673303
NH4NO3 - 14.0932408817721
Secondary OC - 0.3792480413762619
NaNO3 - -2.0913681519396454


# Creating V matrix

In [22]:
#####################################################################################################################################
import pandas as pd
import numpy as np

def calculate_diagonal_matrix(eq_26_17_excel_path, source_contribution_vector, sd_vector):
    # Read the Excel file for original matrix
    df_eq_26_17 = pd.read_excel(eq_26_17_excel_path)

    # Extract data starting from the second column
    data = df_eq_26_17.iloc[:, 1:].values

    # Transpose source_contribution_vector to make it a row vector
    source_contribution_vector_row = source_contribution_vector.T

    # Calculate the sum of squares part
    sum_of_squares_part = np.sum((source_contribution_vector_row**2) * (data**2), axis=1)

    # Calculate the diagonal matrix entries using Eq. 26.17
    diagonal_entries = 1 / (sd_vector**2 + sum_of_squares_part)

    # Create the diagonal matrix
    diagonal_matrix = np.diag(diagonal_entries)

    return diagonal_matrix

source_contribution_vector = source_contributions * 100

sd_vector_excel_path = 'SD 2.5.xlsx'
sd_vector_df = pd.read_excel(sd_vector_excel_path)
sd_vector = sd_vector_df['SD'].values
#####################################################################################################################################

# Calculating V Matrix - Iteration 1 (V1)

In [23]:
# Calculate the diagonal matrix using Eq. 26.17
V1 = calculate_diagonal_matrix(eq_26_17_excel_path, source_contribution_vector, sd_vector)
print("V1 matrix")
print(V1)


V1 matrix
[[7.58965888e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.05056222e-03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.53956290e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.0000000

# Source Profiles as per V1 - (source_contributions2)

In [24]:
source_contributions2 = cmb_solution(A, V1, C)
source = extract_source_names('Matrix A.xlsx')
for i in range(len(source)):
  print(source[i],source_contributions2[i] * 100)

Paved Road Dust [2.1945953]
Vegetative Burning [13.42722583]
Primary Crude Oil [0.28894531]
Motor Vehicles [41.9303625]
Limestone [-0.0700723]
MARINE [-0.26640177]
(NH4)2SO4 [1.6684492]
NH4NO3 [15.8802285]
Secondary OC [-19.25226069]
NaNO3 [-4.03183933]


# Calculating V Matrix - Iteration 2 (V2)

In [25]:
V2 = calculate_diagonal_matrix(eq_26_17_excel_path, source_contributions2, sd_vector)
print(V2)

[[7.65431407e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.52380954e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.60844242e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.

# Source Profiles as per V2 - (source_contributions3)

In [26]:
source_contributions3 = cmb_solution(A, V2, C)
source = extract_source_names('Matrix A.xlsx')
for i in range(len(source)):
  print(source[i],source_contributions3[i] * 100)

Paved Road Dust [1.72761914]
Vegetative Burning [6.75606609]
Primary Crude Oil [0.30916041]
Motor Vehicles [13.86416413]
Limestone [0.01948897]
MARINE [0.09833522]
(NH4)2SO4 [2.94358228]
NH4NO3 [14.35832166]
Secondary OC [-2.20610775]
NaNO3 [-2.37162182]


# Calculating V Matrix - Iteration 3 (V3)

In [27]:
V3 = calculate_diagonal_matrix(eq_26_17_excel_path, source_contributions3, sd_vector)
print(V3)

[[7.65432839e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.03511590e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 6.60846160e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.

# Source Profiles as per V3 - (source_contributions4)

In [28]:
source_contributions4 = cmb_solution(A, V3, C)
source = extract_source_names('Matrix A.xlsx')
for i in range(len(source)):
  print(source[i],source_contributions4[i] * 100)

Paved Road Dust [1.36614096]
Vegetative Burning [6.84548787]
Primary Crude Oil [0.32545473]
Motor Vehicles [11.19986958]
Limestone [0.06626468]
MARINE [0.09466678]
(NH4)2SO4 [3.05250783]
NH4NO3 [14.22581454]
Secondary OC [-0.84843107]
NaNO3 [-2.2313202]


# Source Contribution after a given number of itertions

In [29]:
#####################################################################################################################################
sc = source_contributions
for i in range(10):
  Vi = calculate_diagonal_matrix(eq_26_17_excel_path, sc, sd_vector)
  sc = cmb_solution(A, Vi, C)
  print()
  print("Iteration number : ", i+1)

  for j in range(len(source)):
    print(source[j],sc[j] * 100)
#####################################################################################################################################


Iteration number :  1
Paved Road Dust [1.32169453]
Vegetative Burning [6.97488813]
Primary Crude Oil [0.32791065]
Motor Vehicles [10.10263918]
Limestone [0.07507072]
MARINE [0.08279032]
(NH4)2SO4 [3.09782387]
NH4NO3 [14.17034025]
Secondary OC [-0.35094827]
NaNO3 [-2.17316554]

Iteration number :  2
Paved Road Dust [1.33128576]
Vegetative Burning [6.93598333]
Primary Crude Oil [0.32729228]
Motor Vehicles [10.44219014]
Limestone [0.07276952]
MARINE [0.0866445]
(NH4)2SO4 [3.08372727]
NH4NO3 [14.18759166]
Secondary OC [-0.50459679]
NaNO3 [-2.19125896]

Iteration number :  3
Paved Road Dust [1.33378712]
Vegetative Burning [6.92635942]
Primary Crude Oil [0.32714924]
Motor Vehicles [10.51734722]
Limestone [0.07222162]
MARINE [0.08751821]
(NH4)2SO4 [3.08062457]
NH4NO3 [14.19139276]
Secondary OC [-0.53822791]
NaNO3 [-2.19523891]

Iteration number :  4
Paved Road Dust [1.33435676]
Vegetative Burning [6.92422055]
Primary Crude Oil [0.32711734]
Motor Vehicles [10.53382219]
Limestone [0.07209934]


# Solution of D Part 1

In [31]:
#####################################################################################################################################
# Solving D1
excel_path_W = 'SD 2.5.xlsx' #this matrix contains standard deviations
excel_path_A_D1 = 'D1_A Matrix.xlsx' #this contians the aij matrix
excel_path_C = 'C 2.5.xlsx' #this contians the total contribution of each specie
eq_26_17_excel_path = 'eq 26.17.xlsx' #this contains sigma_aij for V matrix
# Create the weighting matrix
W = create_weighting_matrix(excel_path_W)

# Create the source matrix
A_D1 = create_source_matrix(excel_path_A_D1)

# Create the vector C
C = create_vector_c(excel_path_C)

# Apply the CMB solution
source_contributions_D1 = cmb_solution(A_D1, W, C)
source = extract_source_names('D1_A Matrix.xlsx')
for i in range(len(source)):
  print(source[i],source_contributions_D1[i] * 100)
#####################################################################################################################################

Paved Road Dust [1.29431743]
Vegetative Burning [7.10350566]
Primary Crude Oil [0.33056875]
Motor Vehicles [8.711327]
Limestone [0.08334754]
MARINE [0.06467077]
(NH4)2SO4 [3.1564317]
NH4NO3 [14.09874149]
Secondary OC [0.04541077]
NaNO3 [-2.09786399]


# Solution of D Part 2

In [34]:
#####################################################################################################################################
# Solving D2
excel_path_W = 'SD 2.5.xlsx' #this matrix contains standard deviations
excel_path_A_D2 = 'D2_A Matrix.xlsx' #this contians the aij matrix
excel_path_C = 'C 2.5.xlsx' #this contians the total contribution of each specie
eq_26_17_excel_path = 'eq 26.17.xlsx' #this contains sigma_aij for V matrix
# Create the weighting matrix
W = create_weighting_matrix(excel_path_W)

# Create the source matrix
A_D2 = create_source_matrix(excel_path_A_D2)

# Create the vector C
C = create_vector_c(excel_path_C)

# Apply the CMB solution
source_contributions_D2 = cmb_solution(A_D2, W, C)
source = extract_source_names('D2_A Matrix.xlsx')
for i in range(len(source)):
  print(source[i],source_contributions_D2[i] * 100)
#####################################################################################################################################

Paved Road Dust [1.3070919]
Vegetative Burning [7.00410204]
Primary Crude Oil [0.33021829]
Motor Vehicles [8.78335511]
Limestone [-0.32043525]
MARINE [0.07104099]
(NH4)2SO4 [3.17141727]
NH4NO3 [14.08093553]
Secondary OC [0.29600317]
NaNO3 [-2.07830451]
